In [230]:
import pandas as pd
import os
from pathlib import Path
import pickle
from datetime import datetime, time, timedelta
import re
from pprint import pprint


# Load parameters
params_df = pd.read_csv('Parameter_B120.csv', parse_dates=['from_date', 'to_date'], dayfirst=True)

# Define paths
pickle_path = r"C:\PICKLE"
output_folder = "NRE_processed_csvs"
os.makedirs(output_folder, exist_ok=True)

# Constants
PREFIX = {'SENSEX': 'SX', 'NIFTY': 'Nifty', 'FINNIFTY': 'FN', 'BANKNIFTY': 'BN', 'MIDCPNIFTY': 'MCN', 'BANKEX': 'BX'}
STEPS = {'nifty': 1000, 'banknifty': 5000, 'finnifty': 1000, 'midcpnifty': 1000, 'sensex': 5000, 'bankex': 5000}

max_re_entry = 7

In [231]:
def calculate_stoploss(dataframe, stoploss_price):
    condition = dataframe.high >= stoploss_price
    
    stoploss_hit_index = condition.idxmax() if condition.any() else None
    stoploss_time = dataframe.loc[stoploss_hit_index]["date_time"] if stoploss_hit_index!= None else None
    
    return stoploss_hit_index, stoploss_time



# Function to calculate stop-loss and decay flags
def calculate_reentry_data(option_df, strike, entry_price, sl_price, slippage, max_re_entry):
    
    
    sl0_index, sl0_time = calculate_stoploss(option_df, sl_price)
    sl0_flag = True if sl0_index != None else False
    last_price = option_df['close'].iloc[-1]
    pnl0 = round(entry_price - sl_price - slippage*entry_price if sl0_flag else entry_price - last_price - slippage*entry_price, 2)
    
    is_call_option = "CE" in strike
    
    if is_call_option:
        reentry_data = {
            'CE.Strike' : strike,
            'CE.Price' : entry_price,
            'CE.SL.Price' : sl_price,
            'CE0.SL.Flag' : sl0_flag,
            'CE0.SL.Time' : sl0_time,
            'CE0.PNL' : pnl0
        }
    else:
        reentry_data = {
            'PE.Strike' : strike,
            'PE.Price' : entry_price,
            'PE.SL.Price' : sl_price,
            'PE0.SL.Flag' : sl0_flag,
            'PE0.SL.Time' : sl0_time,
            'PE0.PNL' : pnl0
        }   
    
    continue_flag = False
        
    if sl0_time:
        option_df = option_df[option_df['date_time'] > sl0_time]
    else:
        continue_flag = True
        
            
    for i in range(1, max_re_entry + 1):
        # Check if price returns to entry_price (decay condition)
        decay_flag = False
        decay_time = None
        sl_flag = False
        sl_time = None
        pnl = 0

        if not continue_flag:
            
            # Iterate through the option data to check for decay and stop-loss
            for idx, row in option_df.iterrows():
                if row['low'] <= entry_price and not decay_flag:
                    decay_flag = True
                    decay_time = row['date_time']
                elif decay_flag and row['high'] >= sl_price:
                    sl_flag = True
                    sl_time = row['date_time']
                    pnl = round(entry_price - sl_price - slippage * entry_price, 2)
                    break

            if decay_flag and not sl_flag:
                pnl = round(entry_price - last_price - slippage * entry_price, 2)
                continue_flag = True

        
        if is_call_option:
            # Store results for this re-entry
            reentry_data[f'CE{i}.Decay.Flag'] = decay_flag
            reentry_data[f'CE{i}.Decay.Time'] = decay_time
            reentry_data[f'CE{i}.SL.Flag'] = sl_flag
            reentry_data[f'CE{i}.SL.Time'] = sl_time
            reentry_data[f'CE{i}.PNL'] = pnl
        else:
            # Store results for this re-entry
            reentry_data[f'PE{i}.Decay.Flag'] = decay_flag
            reentry_data[f'PE{i}.Decay.Time'] = decay_time
            reentry_data[f'PE{i}.SL.Flag'] = sl_flag
            reentry_data[f'PE{i}.SL.Time'] = sl_time
            reentry_data[f'PE{i}.PNL'] = pnl
           
        if not continue_flag: 
            if sl_flag:
                option_df = option_df[option_df['date_time'] >= sl_time]
            if not decay_flag:
                continue_flag = True


    return reentry_data
    

In [232]:
# Iterate over each row in the parameters DataFrame
for _, row in params_df.iterrows():
    index = row['index']
    start_date = row['from_date']
    end_date = row['to_date']
    start_time = row['entry_time']
    end_time = row['exit_time']
    orderside = row['orderside']
    method = row['method']
    sl = row['sl']
    ut_sl = row['ut_sl']
    om = row['om']
    gap_price = row['gap_price']
    slippage = row['slippage']

    # Define paths for future and option data
    future_folder = Path(pickle_path) / f"{PREFIX[index]} Future"
    option_folder = Path(pickle_path) / f"{PREFIX[index]} Options"
    
    if index == "NIFTY" or index == "BANKNIFTY" or index == "FINNIFTY" or index == "MIDCPNIFTY" or index == "BANKEX" :
        continue

    # Load future files
    future_files = [file.name for file in future_folder.iterdir() if file.suffix == ".pkl"]
    option_files = [file.name for file in option_folder.iterdir() if file.suffix == ".pkl" and
                    start_date <= datetime.strptime(file.stem[:10], "%Y-%m-%d") <= end_date]
    
    start_time = start_time.zfill(8)
    original_start_time = start_time
    
    count = 0
    all_result = []
    for file in future_files:
        # Load future data
        future_df = pd.read_pickle(f"{future_folder}/{file}")
        time_flag = True
        
        while time_flag:
            future_df = future_df[(future_df.date_time.dt.strftime("%H:%M:%S") >= start_time) & (future_df.date_time.dt.strftime("%H:%M:%S") <= end_time)]
            future_price = future_df['close'].iloc[0]
            entry_time = future_df['date_time'].dt.time.iloc[0]
            step = STEPS[index.lower()]
            target = (future_price // step) * step
            future_date = str(future_df.date_time.dt.date.iloc[0])
            future_day = str(future_df.date_time.dt.day_name().iloc[0])
            start_time = str(entry_time)

            # Calculate X
            X = target * (om / 100) 

            # Load option data
            option_df = pd.read_pickle(option_folder / f"{future_date}_{index.lower()}.pkl")
            option_df = option_df[(option_df.date_time.dt.strftime("%H:%M:%S") >= start_time) & \
                                  (option_df.date_time.dt.strftime("%H:%M:%S") <= end_time)]

            # Filter option data for the start_time
            option_df_time = option_df[option_df['date_time'].dt.strftime("%H:%M:%S") == start_time]
        


            # Find call and put strikes greater than 2X
    #         call_strikes = option_df_time[(option_df_time['scrip'].str.contains('CE')) & (option_df_time['close'] > 2 * X)]
    #         put_strikes = option_df_time[(option_df_time['scrip'].str.contains('PE')) & (option_df_time['close'] > 2 * X)]

            call_scrips = option_df_time[(option_df_time['scrip'].str.contains('CE')) & (option_df_time['close'] >= X)]
            put_scrips = option_df_time[(option_df_time['scrip'].str.contains('PE')) & (option_df_time['close'] >= X)]

            if call_scrips.empty or put_scrips.empty:
                current_time = datetime.strptime(start_time, "%H:%M:%S")
                new_time = (current_time + timedelta(minutes=1)).time()
                start_time = new_time.strftime("%H:%M:%S")
            else:
                time_flag = False
            

        # Get the first call and put strikes greater than 2X
#         call_strike = call_strikes['scrip'].iloc[0]
#         put_strike = put_strikes['scrip'].iloc[0]

        call_scrip = call_scrips['scrip'].iloc[0]
        put_scrip = put_scrips['scrip'].iloc[0]
        
        call_strike = int(re.findall(r"\d{5}", call_scrip)[0])
        put_strike = int(re.findall(r"\d{5}", put_scrip)[0])
        
        # Generate pairs
        pairs = [
            (call_strike, put_strike),
            (call_strike, put_strike + gap_price),
            (call_strike, put_strike - gap_price),
            (call_strike + gap_price, put_strike),
            (call_strike - gap_price, put_strike)
        ]
    
        
        # Calculate sum and difference for each pair
        pair_data = []
        for pair in pairs:
            call_price = option_df_time[(option_df_time['scrip'].str.contains('CE')) & (option_df_time['scrip'].str.contains(str(pair[0])))]['close'].iloc[0] if not option_df_time[(option_df_time['scrip'].str.contains('CE')) & (option_df_time['scrip'].str.contains(str(pair[0])))].empty else None
            put_price = option_df_time[(option_df_time['scrip'].str.contains('PE')) & (option_df_time['scrip'].str.contains(str(pair[1])))]['close'].iloc[0] if not option_df_time[(option_df_time['scrip'].str.contains('PE')) & (option_df_time['scrip'].str.contains(str(pair[1])))].empty else None
            if call_price == None or put_price == None:
                continue
            total = call_price + put_price
            difference = abs(call_price - put_price)
            pair_data.append({'pair': pair, 'total': total, 'difference': difference})
        
        # Filter pairs based on total value
        filtered_pairs = [pair for pair in pair_data if 2 * X <= pair['total'] <= 3 * X]

        if not filtered_pairs:
            print("filteres_pairs not found hence skip")
            continue
            
            
        # Select the pair with the smallest difference
        final_pair = min(filtered_pairs, key=lambda x: x['difference'])
        
        dte_df = pd.read_csv('DTE.csv')
        formatted_date = datetime.strptime(future_date, "%Y-%m-%d").strftime("%d-%m-%Y")
        this_dte = int(dte_df.loc[dte_df["Date"] == formatted_date, f'{index}'].values[0])
        
        ce_strike = f"{final_pair['pair'][0]}CE"
        ce_price = option_df_time[option_df_time.scrip == ce_strike ]['close'].iloc[0]
        ce_sl_price = ce_price * (sl/100 +1)
        
        pe_strike = f"{final_pair['pair'][1]}PE"
        pe_price = option_df_time[option_df_time.scrip == pe_strike ]['close'].iloc[0]
        pe_sl_price = pe_price * (sl/100 +1)
        
    
        # Save the results
        result = {
            'P_Strategy' : "NRE",
            'P_Index' : index,
            'P_StartTime' : start_time,
            'P_EndTime' : end_time,
            'P_OrderSide' : orderside,
            'P_Method' : method,
            'P_SL' : sl,
            'P_ReSL' : sl,
            'P_OM' : om,
            'Date' : future_date,
            'Day' : future_day,
            'DTE' : this_dte,
            'EntryTime' : entry_time,
            'Future' : future_price
        }
        
        # Calculate re-entry data for call option
        call_reentry_data = calculate_reentry_data(option_df[option_df.scrip == ce_strike], ce_strike, ce_price, ce_sl_price, slippage, max_re_entry)

        # Calculate re-entry data for put option
        put_reentry_data = calculate_reentry_data(option_df[option_df.scrip == pe_strike], pe_strike, pe_price, pe_sl_price, slippage, max_re_entry)
        
        pprint(call_reentry_data)
        print("dfsd")
        pprint(put_reentry_data)
        # Merge call and put re-entry data into results
        result.update(call_reentry_data)
        result.update(put_reentry_data)
        
        count+=1
        print(count)
        print()
        print()
        print()
        
        
        start_time = original_start_time
        
        # Append the result to the list
        all_result.append(result)
        
        if count == 100:
            break
        
    # Convert results to DataFrame
    result_df = pd.DataFrame(all_result)


    # Save results to CSV
    output_csv_path = Path(output_folder) / f"{future_date}_{index}_results.csv"
    result_df.to_csv(output_csv_path, index=False)

    print(f"Results saved to {output_csv_path}")
    
    result_df
    


{'CE.Price': np.float64(687.0),
 'CE.SL.Price': np.float64(824.4),
 'CE.Strike': '71800CE',
 'CE0.PNL': np.float64(-145.99),
 'CE0.SL.Flag': True,
 'CE0.SL.Time': Timestamp('2024-01-01 13:30:00'),
 'CE1.Decay.Flag': True,
 'CE1.Decay.Time': Timestamp('2024-01-01 15:22:00'),
 'CE1.PNL': np.float64(-1.69),
 'CE1.SL.Flag': False,
 'CE1.SL.Time': None,
 'CE2.Decay.Flag': False,
 'CE2.Decay.Time': None,
 'CE2.PNL': 0,
 'CE2.SL.Flag': False,
 'CE2.SL.Time': None,
 'CE3.Decay.Flag': False,
 'CE3.Decay.Time': None,
 'CE3.PNL': 0,
 'CE3.SL.Flag': False,
 'CE3.SL.Time': None,
 'CE4.Decay.Flag': False,
 'CE4.Decay.Time': None,
 'CE4.PNL': 0,
 'CE4.SL.Flag': False,
 'CE4.SL.Time': None,
 'CE5.Decay.Flag': False,
 'CE5.Decay.Time': None,
 'CE5.PNL': 0,
 'CE5.SL.Flag': False,
 'CE5.SL.Time': None,
 'CE6.Decay.Flag': False,
 'CE6.Decay.Time': None,
 'CE6.PNL': 0,
 'CE6.SL.Flag': False,
 'CE6.SL.Time': None,
 'CE7.Decay.Flag': False,
 'CE7.Decay.Time': None,
 'CE7.PNL': 0,
 'CE7.SL.Flag': False,
 'CE7

{'CE.Price': np.float64(726.05),
 'CE.SL.Price': np.float64(871.2599999999999),
 'CE.Strike': '71700CE',
 'CE0.PNL': np.float64(451.87),
 'CE0.SL.Flag': False,
 'CE0.SL.Time': None,
 'CE1.Decay.Flag': False,
 'CE1.Decay.Time': None,
 'CE1.PNL': 0,
 'CE1.SL.Flag': False,
 'CE1.SL.Time': None,
 'CE2.Decay.Flag': False,
 'CE2.Decay.Time': None,
 'CE2.PNL': 0,
 'CE2.SL.Flag': False,
 'CE2.SL.Time': None,
 'CE3.Decay.Flag': False,
 'CE3.Decay.Time': None,
 'CE3.PNL': 0,
 'CE3.SL.Flag': False,
 'CE3.SL.Time': None,
 'CE4.Decay.Flag': False,
 'CE4.Decay.Time': None,
 'CE4.PNL': 0,
 'CE4.SL.Flag': False,
 'CE4.SL.Time': None,
 'CE5.Decay.Flag': False,
 'CE5.Decay.Time': None,
 'CE5.PNL': 0,
 'CE5.SL.Flag': False,
 'CE5.SL.Time': None,
 'CE6.Decay.Flag': False,
 'CE6.Decay.Time': None,
 'CE6.PNL': 0,
 'CE6.SL.Flag': False,
 'CE6.SL.Time': None,
 'CE7.Decay.Flag': False,
 'CE7.Decay.Time': None,
 'CE7.PNL': 0,
 'CE7.SL.Flag': False,
 'CE7.SL.Time': None}
dfsd
{'PE.Price': np.float64(926.0),
 'PE

{'CE.Price': np.float64(756.2),
 'CE.SL.Price': np.float64(907.44),
 'CE.Strike': '72800CE',
 'CE0.PNL': np.float64(-80.5),
 'CE0.SL.Flag': False,
 'CE0.SL.Time': None,
 'CE1.Decay.Flag': False,
 'CE1.Decay.Time': None,
 'CE1.PNL': 0,
 'CE1.SL.Flag': False,
 'CE1.SL.Time': None,
 'CE2.Decay.Flag': False,
 'CE2.Decay.Time': None,
 'CE2.PNL': 0,
 'CE2.SL.Flag': False,
 'CE2.SL.Time': None,
 'CE3.Decay.Flag': False,
 'CE3.Decay.Time': None,
 'CE3.PNL': 0,
 'CE3.SL.Flag': False,
 'CE3.SL.Time': None,
 'CE4.Decay.Flag': False,
 'CE4.Decay.Time': None,
 'CE4.PNL': 0,
 'CE4.SL.Flag': False,
 'CE4.SL.Time': None,
 'CE5.Decay.Flag': False,
 'CE5.Decay.Time': None,
 'CE5.PNL': 0,
 'CE5.SL.Flag': False,
 'CE5.SL.Time': None,
 'CE6.Decay.Flag': False,
 'CE6.Decay.Time': None,
 'CE6.PNL': 0,
 'CE6.SL.Flag': False,
 'CE6.SL.Time': None,
 'CE7.Decay.Flag': False,
 'CE7.Decay.Time': None,
 'CE7.PNL': 0,
 'CE7.SL.Flag': False,
 'CE7.SL.Time': None}
dfsd
{'PE.Price': np.float64(779.15),
 'PE.SL.Price': 

{'CE.Price': np.float64(739.4),
 'CE.SL.Price': np.float64(887.28),
 'CE.Strike': '71600CE',
 'CE0.PNL': np.float64(269.01),
 'CE0.SL.Flag': False,
 'CE0.SL.Time': None,
 'CE1.Decay.Flag': False,
 'CE1.Decay.Time': None,
 'CE1.PNL': 0,
 'CE1.SL.Flag': False,
 'CE1.SL.Time': None,
 'CE2.Decay.Flag': False,
 'CE2.Decay.Time': None,
 'CE2.PNL': 0,
 'CE2.SL.Flag': False,
 'CE2.SL.Time': None,
 'CE3.Decay.Flag': False,
 'CE3.Decay.Time': None,
 'CE3.PNL': 0,
 'CE3.SL.Flag': False,
 'CE3.SL.Time': None,
 'CE4.Decay.Flag': False,
 'CE4.Decay.Time': None,
 'CE4.PNL': 0,
 'CE4.SL.Flag': False,
 'CE4.SL.Time': None,
 'CE5.Decay.Flag': False,
 'CE5.Decay.Time': None,
 'CE5.PNL': 0,
 'CE5.SL.Flag': False,
 'CE5.SL.Time': None,
 'CE6.Decay.Flag': False,
 'CE6.Decay.Time': None,
 'CE6.PNL': 0,
 'CE6.SL.Flag': False,
 'CE6.SL.Time': None,
 'CE7.Decay.Flag': False,
 'CE7.Decay.Time': None,
 'CE7.PNL': 0,
 'CE7.SL.Flag': False,
 'CE7.SL.Time': None}
dfsd
{'PE.Price': np.float64(762.95),
 'PE.SL.Price':

{'CE.Price': np.float64(736.6),
 'CE.SL.Price': np.float64(883.92),
 'CE.Strike': '72100CE',
 'CE0.PNL': np.float64(414.54),
 'CE0.SL.Flag': False,
 'CE0.SL.Time': None,
 'CE1.Decay.Flag': False,
 'CE1.Decay.Time': None,
 'CE1.PNL': 0,
 'CE1.SL.Flag': False,
 'CE1.SL.Time': None,
 'CE2.Decay.Flag': False,
 'CE2.Decay.Time': None,
 'CE2.PNL': 0,
 'CE2.SL.Flag': False,
 'CE2.SL.Time': None,
 'CE3.Decay.Flag': False,
 'CE3.Decay.Time': None,
 'CE3.PNL': 0,
 'CE3.SL.Flag': False,
 'CE3.SL.Time': None,
 'CE4.Decay.Flag': False,
 'CE4.Decay.Time': None,
 'CE4.PNL': 0,
 'CE4.SL.Flag': False,
 'CE4.SL.Time': None,
 'CE5.Decay.Flag': False,
 'CE5.Decay.Time': None,
 'CE5.PNL': 0,
 'CE5.SL.Flag': False,
 'CE5.SL.Time': None,
 'CE6.Decay.Flag': False,
 'CE6.Decay.Time': None,
 'CE6.PNL': 0,
 'CE6.SL.Flag': False,
 'CE6.SL.Time': None,
 'CE7.Decay.Flag': False,
 'CE7.Decay.Time': None,
 'CE7.PNL': 0,
 'CE7.SL.Flag': False,
 'CE7.SL.Time': None}
dfsd
{'PE.Price': np.float64(735.0),
 'PE.SL.Price': 

...

{'CE.Price': np.float64(704.75),
 'CE.SL.Price': np.float64(845.6999999999999),
 'CE.Strike': '73400CE',
 'CE0.PNL': np.float64(96.74),
 'CE0.SL.Flag': False,
 'CE0.SL.Time': None,
 'CE1.Decay.Flag': False,
 'CE1.Decay.Time': None,
 'CE1.PNL': 0,
 'CE1.SL.Flag': False,
 'CE1.SL.Time': None,
 'CE2.Decay.Flag': False,
 'CE2.Decay.Time': None,
 'CE2.PNL': 0,
 'CE2.SL.Flag': False,
 'CE2.SL.Time': None,
 'CE3.Decay.Flag': False,
 'CE3.Decay.Time': None,
 'CE3.PNL': 0,
 'CE3.SL.Flag': False,
 'CE3.SL.Time': None,
 'CE4.Decay.Flag': False,
 'CE4.Decay.Time': None,
 'CE4.PNL': 0,
 'CE4.SL.Flag': False,
 'CE4.SL.Time': None,
 'CE5.Decay.Flag': False,
 'CE5.Decay.Time': None,
 'CE5.PNL': 0,
 'CE5.SL.Flag': False,
 'CE5.SL.Time': None,
 'CE6.Decay.Flag': False,
 'CE6.Decay.Time': None,
 'CE6.PNL': 0,
 'CE6.SL.Flag': False,
 'CE6.SL.Time': None,
 'CE7.Decay.Flag': False,
 'CE7.Decay.Time': None,
 'CE7.PNL': 0,
 'CE7.SL.Flag': False,
 'CE7.SL.Time': None}
dfsd
{'PE.Price': np.float64(704.2),
 'PE.

{'CE.Price': np.float64(776.55),
 'CE.SL.Price': np.float64(931.8599999999999),
 'CE.Strike': '74200CE',
 'CE0.PNL': np.float64(-165.02),
 'CE0.SL.Flag': True,
 'CE0.SL.Time': Timestamp('2024-04-30 09:54:00'),
 'CE1.Decay.Flag': True,
 'CE1.Decay.Time': Timestamp('2024-04-30 14:51:00'),
 'CE1.PNL': np.float64(155.04),
 'CE1.SL.Flag': False,
 'CE1.SL.Time': None,
 'CE2.Decay.Flag': False,
 'CE2.Decay.Time': None,
 'CE2.PNL': 0,
 'CE2.SL.Flag': False,
 'CE2.SL.Time': None,
 'CE3.Decay.Flag': False,
 'CE3.Decay.Time': None,
 'CE3.PNL': 0,
 'CE3.SL.Flag': False,
 'CE3.SL.Time': None,
 'CE4.Decay.Flag': False,
 'CE4.Decay.Time': None,
 'CE4.PNL': 0,
 'CE4.SL.Flag': False,
 'CE4.SL.Time': None,
 'CE5.Decay.Flag': False,
 'CE5.Decay.Time': None,
 'CE5.PNL': 0,
 'CE5.SL.Flag': False,
 'CE5.SL.Time': None,
 'CE6.Decay.Flag': False,
 'CE6.Decay.Time': None,
 'CE6.PNL': 0,
 'CE6.SL.Flag': False,
 'CE6.SL.Time': None,
 'CE7.Decay.Flag': False,
 'CE7.Decay.Time': None,
 'CE7.PNL': 0,
 'CE7.SL.Flag'

{'CE.Price': np.float64(705.9),
 'CE.SL.Price': np.float64(847.0799999999999),
 'CE.Strike': '72800CE',
 'CE0.PNL': np.float64(-150.0),
 'CE0.SL.Flag': True,
 'CE0.SL.Time': Timestamp('2024-05-08 10:09:00'),
 'CE1.Decay.Flag': True,
 'CE1.Decay.Time': Timestamp('2024-05-08 10:21:00'),
 'CE1.PNL': np.float64(-150.0),
 'CE1.SL.Flag': True,
 'CE1.SL.Time': Timestamp('2024-05-08 11:47:00'),
 'CE2.Decay.Flag': False,
 'CE2.Decay.Time': None,
 'CE2.PNL': 0,
 'CE2.SL.Flag': False,
 'CE2.SL.Time': None,
 'CE3.Decay.Flag': False,
 'CE3.Decay.Time': None,
 'CE3.PNL': 0,
 'CE3.SL.Flag': False,
 'CE3.SL.Time': None,
 'CE4.Decay.Flag': False,
 'CE4.Decay.Time': None,
 'CE4.PNL': 0,
 'CE4.SL.Flag': False,
 'CE4.SL.Time': None,
 'CE5.Decay.Flag': False,
 'CE5.Decay.Time': None,
 'CE5.PNL': 0,
 'CE5.SL.Flag': False,
 'CE5.SL.Time': None,
 'CE6.Decay.Flag': False,
 'CE6.Decay.Time': None,
 'CE6.PNL': 0,
 'CE6.SL.Flag': False,
 'CE6.SL.Time': None,
 'CE7.Decay.Flag': False,
 'CE7.Decay.Time': None,
 'CE

{'CE.Price': np.float64(744.1),
 'CE.SL.Price': np.float64(892.92),
 'CE.Strike': '72700CE',
 'CE0.PNL': np.float64(128.6),
 'CE0.SL.Flag': False,
 'CE0.SL.Time': None,
 'CE1.Decay.Flag': False,
 'CE1.Decay.Time': None,
 'CE1.PNL': 0,
 'CE1.SL.Flag': False,
 'CE1.SL.Time': None,
 'CE2.Decay.Flag': False,
 'CE2.Decay.Time': None,
 'CE2.PNL': 0,
 'CE2.SL.Flag': False,
 'CE2.SL.Time': None,
 'CE3.Decay.Flag': False,
 'CE3.Decay.Time': None,
 'CE3.PNL': 0,
 'CE3.SL.Flag': False,
 'CE3.SL.Time': None,
 'CE4.Decay.Flag': False,
 'CE4.Decay.Time': None,
 'CE4.PNL': 0,
 'CE4.SL.Flag': False,
 'CE4.SL.Time': None,
 'CE5.Decay.Flag': False,
 'CE5.Decay.Time': None,
 'CE5.PNL': 0,
 'CE5.SL.Flag': False,
 'CE5.SL.Time': None,
 'CE6.Decay.Flag': False,
 'CE6.Decay.Time': None,
 'CE6.PNL': 0,
 'CE6.SL.Flag': False,
 'CE6.SL.Time': None,
 'CE7.Decay.Flag': False,
 'CE7.Decay.Time': None,
 'CE7.PNL': 0,
 'CE7.SL.Flag': False,
 'CE7.SL.Time': None}
dfsd
{'PE.Price': np.float64(716.3),
 'PE.SL.Price': n

{'CE.Price': np.float64(1324.15),
 'CE.SL.Price': np.float64(1588.98),
 'CE.Strike': '73000CE',
 'CE0.PNL': np.float64(-281.38),
 'CE0.SL.Flag': True,
 'CE0.SL.Time': Timestamp('2024-05-23 10:20:00'),
 'CE1.Decay.Flag': False,
 'CE1.Decay.Time': None,
 'CE1.PNL': 0,
 'CE1.SL.Flag': False,
 'CE1.SL.Time': None,
 'CE2.Decay.Flag': False,
 'CE2.Decay.Time': None,
 'CE2.PNL': 0,
 'CE2.SL.Flag': False,
 'CE2.SL.Time': None,
 'CE3.Decay.Flag': False,
 'CE3.Decay.Time': None,
 'CE3.PNL': 0,
 'CE3.SL.Flag': False,
 'CE3.SL.Time': None,
 'CE4.Decay.Flag': False,
 'CE4.Decay.Time': None,
 'CE4.PNL': 0,
 'CE4.SL.Flag': False,
 'CE4.SL.Time': None,
 'CE5.Decay.Flag': False,
 'CE5.Decay.Time': None,
 'CE5.PNL': 0,
 'CE5.SL.Flag': False,
 'CE5.SL.Time': None,
 'CE6.Decay.Flag': False,
 'CE6.Decay.Time': None,
 'CE6.PNL': 0,
 'CE6.SL.Flag': False,
 'CE6.SL.Time': None,
 'CE7.Decay.Flag': False,
 'CE7.Decay.Time': None,
 'CE7.PNL': 0,
 'CE7.SL.Flag': False,
 'CE7.SL.Time': None}
dfsd
{'PE.Price': np.f

In [222]:
call_scrips

,index,scrip,date_time,open,high,low,close,volume,openinterest,expiry_date,dte
426,SENSEX,71200CE,2024-01-03 09:20:00,700.00,700.00,700.00,700.00,40,0,2024-01-05,2
428,SENSEX,71100CE,2024-01-03 09:20:00,784.05,784.05,784.05,784.05,10,0,2024-01-05,2
431,SENSEX,70800CE,2024-01-03 09:20:00,1034.60,1034.60,1034.60,1034.60,10,0,2024-01-05,2


In [223]:
put_scrips

,index,scrip,date_time,open,high,low,close,volume,openinterest,expiry_date,dte
427,SENSEX,72300PE,2024-01-03 09:20:00,685.75,725.95,685.75,716.35,630,0,2024-01-05,2
429,SENSEX,72400PE,2024-01-03 09:20:00,773.00,791.80,760.25,791.80,100,0,2024-01-05,2
430,SENSEX,72500PE,2024-01-03 09:20:00,853.00,880.85,841.20,872.30,1290,0,2024-01-05,2
432,SENSEX,72900PE,2024-01-03 09:20:00,1201.25,1201.25,1201.25,1201.25,10,0,2024-01-05,2
433,SENSEX,73000PE,2024-01-03 09:20:00,1293.95,1293.95,1293.95,1293.95,10,0,2024-01-05,2


In [164]:
option_df_time

,index,scrip,date_time,open,high,low,close,volume,openinterest,expiry_date,dte
254,SENSEX,69000PE,2024-01-08 09:20:00,7.00,7.00,7.00,7.00,600,0,2024-01-12,4
255,SENSEX,74500CE,2024-01-08 09:20:00,13.00,13.00,13.00,13.00,10,0,2024-01-12,4
256,SENSEX,74300CE,2024-01-08 09:20:00,20.00,20.00,20.00,20.00,10,0,2024-01-12,4
257,SENSEX,74000CE,2024-01-08 09:20:00,25.70,26.00,24.45,24.45,1510,0,2024-01-12,4
258,SENSEX,74100CE,2024-01-08 09:20:00,25.60,25.60,23.00,25.55,70,0,2024-01-12,4
259,SENSEX,70000PE,2024-01-08 09:20:00,28.25,29.00,28.25,29.00,540,0,2024-01-12,4
260,SENSEX,73900CE,2024-01-08 09:20:00,32.40,32.40,32.00,32.00,120,0,2024-01-12,4
261,SENSEX,73800CE,2024-01-08 09:20:00,32.80,33.25,32.80,33.25,120,0,2024-01-12,4
262,SENSEX,70100PE,2024-01-08 09:20:00,35.65,36.55,35.65,36.55,90,0,2024-01-12,4
263,SENSEX,70200PE,2024-01-08 09:20:00,37.05,37.30,37.00,37.00,110,0,2024-01-12,4
